In [1]:
# Tensorflow 2.3でメモリを指定及び節約して使うためのおまじない。
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[4], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[4], True)

import numpy as np
import random

np.random.seed(219)
random.seed(219)
tf.random.set_seed(219)


In [2]:
import os
dirpath = "/workdir/taki_lab/tiny-imagenet-200/"
train_dir = os.path.join(dirpath, "train")
val_dir = os.path.join(dirpath, "val")
test_dir = os.path.join(dirpath, "test")

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

N_TRAIN = 100000
N_VAL = 10000
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                              rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest")

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.flow_from_directory(train_dir,
                                               target_size=INPUT_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

val_generator = val_gen.flow_from_directory(val_dir,
                                           target_size=INPUT_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [4]:
from keras import layers, Model

class CBR(Model):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()
        params = {
            "filters":filters,
            "kernel_size":kernel_size,
            "strides":strides,
            "padding":"same",
            "use_bias":True,
            "kernel_initializer":"he_normal"
        }
        
        self.layers_ = [
            layers.Conv2D(**params),
            layers.ReLU(),
            layers.BatchNormalization()
        ]
        
    def call(self, inputs):
        for layer in self.layers_:
            inputs = layer(inputs)
        return inputs

class VGG16(Model):
    def __init__(self, output_size):
        super().__init__()
        self.layers_ = [
            CBR(64, 3, 1),
            CBR(64, 3, 1),
            layers.MaxPool2D((2,2)),
            CBR(128, 3, 1),
            CBR(128, 3, 1),
            layers.MaxPool2D((2,2)),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            layers.MaxPool2D((2,2)),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            layers.MaxPool2D((2,2)),
            CBR(512, 3, 1),
            CBR(512, 3, 1),
            CBR(512, 3, 1),
            layers.MaxPool2D((2,2)),
            layers.GlobalAveragePooling2D(),
            layers.Dense(4096, kernel_initializer="he_normal"),
            layers.Dense(output_size, activation="softmax")
        ]
    
    def call(self,inputs):
        for layer in self.layers_:
            inputs = layer(inputs)
        return inputs

In [ ]:
from keras import optimizers
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# wandbの設定
wandb.init(project="tiny-image-net", name="vgg16_normal")
config = wandb.config
config.learning_rate = 0.01
config.lr_factor = 0.5
config.optimaizer = "Adam"
config.batch_size = BATCH_SIZE
config.input_size = INPUT_SIZE
config.rotation_range = 10
config.width_shift_range = 0.1
config.height_shift_range = 0.1
config.shear_range = 0.1
config.zoom_range = 0.1

# 最適化設定

model = VGG16(200)
model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
             loss="categorical_crossentropy",
             metrics=["acc"])

early_stopping = EarlyStopping(monitor="val_acc", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

callback_list = [early_stopping, reduce_lr, WandbCallback()]


# 学習
history = model.fit(train_generator,
    steps_per_epoch=N_TRAIN//BATCH_SIZE ,
    epochs=1000,
    validation_data=val_generator,
    validation_steps=N_VAL//BATCH_SIZE,
    use_multiprocessing=True,
    workers=24,
    callbacks=callback_list)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1000
3125/3125 [==============================] - ETA: 0s - loss: 6.7405 - acc: 0.0219WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


3125/3125 [==============================] - 835s 267ms/step - loss: 6.7405 - acc: 0.0219 - val_loss: 6.1659 - val_acc: 0.0232
Epoch 2/1000
3125/3125 [==============================] - 841s 269ms/step - loss: 5.1955 - acc: 0.0315 - val_loss: 36872704.0000 - val_acc: 0.0280
Epoch 3/1000
3125/3125 [==============================] - 828s 265ms/step - loss: 5.1069 - acc: 0.0505 - val_loss: 168835.5938 - val_acc: 0.0321
Epoch 4/1000
3125/3125 [==============================] - 833s 266ms/step - loss: 4.1206 - acc: 0.1151 - val_loss: 103.8786 - val_acc: 0.1031
Epoch 5/1000
3125/3125 [==============================] - 834s 267ms/step - loss: 3.9721 - acc: 0.1388 - val_loss: 5.0514 - val_acc: 0.1021
Epoch 6/1000
3125/3125 [==============================] - 831s 266ms/step - loss: 3.8252 - acc: 0.1603 - val_loss: 3817.2583 - val_acc: 0.1716
Epoch 7/1000
3125/3125 [==============================] - 827s 265ms/step - loss: 3.6877 - acc: 0.1821 - val_loss: 9336.4805 - val_acc: 0.2030
Epoch 8/1000
